In [ ]:
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
import polars as pl

In [ ]:
beauty_review: Dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_review_All_Beauty",
    split="full",
    trust_remote_code=True,
)

beauty_meta: Dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "raw_meta_All_Beauty",
    split="full",
    trust_remote_code=True,
)

In [ ]:
beauty_review.to_parquet("amazon_review_beauty.parquet")
beauty_meta.to_parquet("amazon_meta_beauty.parquet")

In [ ]:
df_review: pl.DataFrame = pl.read_parquet("amazon_review_beauty.parquet")
df_meta: pl.DataFrame = pl.read_parquet("amazon_meta_beauty.parquet")

In [20]:
df: pl.DataFrame = df_review.join(df_meta, on="parent_asin", how="left")

In [21]:
def extract_brand() -> pl.Expr:
    store_clean: pl.Then = pl.when(pl.col("store").str.strip_chars() != "").then(pl.col("store"))
    details_brand: pl.Expr = pl.col("details").str.extract(r"Brand[:\s]*([^\s,;]+)", 1)
    
    return pl.coalesce([
        store_clean,
        details_brand,
        pl.lit("Unknown")
    ]).alias("brand")

In [22]:
# Drop invalid ratings (missing or not in [1,2,3,4,5])
df: pl.DataFrame = df.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))
# Drop empty review texts
df = df.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)

df = df.with_columns([extract_brand()])

df = df.with_columns([
    pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
    (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
])


df = df.unique(subset=["user_id", "text", "asin"], keep="first")

In [23]:
df

rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,title_right,average_rating,rating_number,features,description,price,images_right,videos,store,categories,details,bought_together,subtitle,author,brand,review_length,year
f64,str,str,list[struct[4]],str,str,str,i64,i64,bool,str,str,f64,i64,list[str],list[str],str,struct[4],struct[3],str,list[str],str,str,str,str,str,u32,i32
5.0,"""Buy them""","""The are the best hair ties. Ne…",[],"""B07V5CW5CG""","""B07V5CW5CG""","""AGC73F2KHTUOZWUDOZYGTA5G6Z2Q""",1580159609883,0,true,"""All Beauty""","""WillingTee 30pcs Metal Color S…",4.2,34,[],[],"""None""","{[""https://m.media-amazon.com/images/I/71tV2goA1QL._SL1200_.jpg"", ""https://m.media-amazon.com/images/I/71oK2Y386iL._SL1200_.jpg""],[""https://m.media-amazon.com/images/I/51NFugypGcL.jpg"", ""https://m.media-amazon.com/images/I/51+nJXXbQ4L.jpg""],[""https://m.media-amazon.com/images/I/51NFugypGcL._SS40_.jpg"", ""https://m.media-amazon.com/images/I/51+nJXXbQ4L._SS40_.jpg""],[""MAIN"", ""PT01""]}","{[],[],[]}","""WillingTee""",[],"""{""Material"": ""Plastic"", ""Item …",null,null,null,"""WillingTee""",17,2020
1.0,"""It Rusted some Implements""","""I decide as I wanted to try a …",[],"""B06XGWJJFW""","""B06XGWJJFW""","""AFMR6XY4DH2BONH375ZTFRKK3YVQ""",1600970983764,2,true,"""All Beauty""","""MOD Clean""",4.4,193,[],[],"""17.13""","{[null],[""https://m.media-amazon.com/images/I/31dH31wdMjL.jpg""],[""https://m.media-amazon.com/images/I/31dH31wdMjL._SS40_.jpg""],[""MAIN""]}","{[],[],[]}","""Artist Unknown""",[],"""{""Brand"": ""Artist Unknown"", ""I…",null,null,null,"""Artist Unknown""",45,2020
5.0,"""OK2BUY""","""Beats Walmart price, great pro…",[],"""B00JISAUR8""","""B00JISAUR8""","""AGZTW2YLNOPFKCT4HPCYNLC74JVA""",1495754871000,0,true,"""All Beauty""","""Garnier Fructis Sleek & Shine …",4.1,64,"[""Leave-in conditioning cream for up to 3 day sleek"", ""Intensely smooth"", ""Proven strength system""]","[""Garnier Fructis Sleek & Shine Leave-In Conditioning Cream, 10.2 oz, 2 pk""]","""8.98""","{[""https://m.media-amazon.com/images/I/71sxKqdPecL._SL1500_.jpg"", ""https://m.media-amazon.com/images/I/71WpuymxM8L._SL1500_.jpg"", … ""https://m.media-amazon.com/images/I/8133DyePyaL._SL1500_.jpg""],[""https://m.media-amazon.com/images/I/41gUHu-MCzL.jpg"", ""https://m.media-amazon.com/images/I/41NlRHsEsPL.jpg"", … ""https://m.media-amazon.com/images/I/51y971sKo+L.jpg""],[""https://m.media-amazon.com/images/I/41gUHu-MCzL._SS40_.jpg"", ""https://m.media-amazon.com/images/I/41NlRHsEsPL._SS40_.jpg"", … ""https://m.media-amazon.com/images/I/51y971sKo+L._SS40_.jpg""],[""MAIN"", ""PT01"", … ""DTLS""]}","{[""Here's The Secret To Having Beautiful Hair! MUST WATCH!"", ""Style Code Live: Redbook Garnier Integration | Pro Tips for Perfect Hair | Look 2""],[""https://www.amazon.com/vdp/0024af0ae1234a3fac9ffd7460ead76b?ref=dp_vse_rvc_0"", ""https://www.amazon.com/vdp/afdc73b298dc4ca0acb1796a6f0bd11d?ref=dp_vse_rvc_1""],[""/shop/influencer-98d451b5"", """"]}","""Garnier""",[],"""{""Brand"": ""Garnier"", ""Item For…",null,null,null,"""Garnier""",5,2017
5.0,"""I’m in love & Thank You""","""WOW! I NEVER write reviews, bu…","[{""IMAGE"",""https://images-na.ssl-images-amazon.com/images/I/71UDVzQRTXL.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71UDVzQRTXL._SL800_.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71UDVzQRTXL._SL256_.jpg""}, {""IMAGE"",""https://images-na.ssl-images-amazon.com/images/I/71hVCiPD0FL.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71hVCiPD0FL._SL800_.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71hVCiPD0FL._SL256_.jpg""}, {""IMAGE"",""https://images-na.ssl-images-amazon.com/images/I/71unZzeJlGL.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71unZzeJlGL._SL800_.jpg"",""https://images-na.ssl-images-amazon.com/images/I/71unZzeJlGL._SL256_.jpg""}]","""B083NVC5RB""","""B083NVC5RB""","""AHREBJPMMF3WUWOUL6OEBCAPTXYA""",1587313701297,10,true,"""